In [1]:
import sys
sys.path.append("../src")

from dataloaders import ECGDataset

# Example of usage
path = '../data/physionet.org/files/ptb-xl/1.0.2/'
sampling_rate = 100  # or other value depending on your data




In [2]:
import pandas as pd
import ast
import wfdb
import numpy as np

def aggregate_diagnostic(y_dic) -> list:
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

def load_raw_data(df: pd.DataFrame, sampling_rate: int, path: str) -> np.array:
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    return np.array([signal for signal, meta in data])


Y = pd.read_csv(path + 'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

agg_df = pd.read_csv(path + 'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

print("[INFO] Obtaining diagnostic_superclass ...")
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

super_classes = [x[0] if len(x) > 0 else '' for x in Y['diagnostic_superclass'].values.tolist()]
unique_superclasses = list(set(super_classes))

X = load_raw_data(Y, sampling_rate, path)

# # take all the samples that are in the take_folds
# X = X[np.isin(Y.strat_fold.values, take_folds)]
# super_classes = [x for i, x in enumerate(super_classes) if Y.strat_fold.values[i] in take_folds]

[INFO] Obtaining diagnostic_superclass ...


In [3]:

# save X and super_classes to "data_dumps/sr100" as numpy objects
import numpy as np


for fold in range(1,11):
    
    X_fold = X[np.isin(Y.strat_fold.values, [fold])].copy()
    super_classes_fold = [x for i, x in enumerate(super_classes) if Y.strat_fold.values[i] in [fold]]
    
    np.save(f'../data/data_dumps/sr100/X_fold_{fold}.npy', X_fold)
    np.save(f'../data/data_dumps/sr100/superclasses_{fold}.npy', super_classes_fold)
    